In [2]:
from faker import Faker
import random
import pandas as pd

faker = Faker()

In [3]:
# 사용자 데이터 생성
users = []
for i in range(1, 101):
    user = {
        "user_id": i,
        "age": random.randint(18, 65),
        "gender": random.choice(["Male", "Female"]),
        "team_preference":random.choice(["기아 타이거즈", "삼성 라이온즈", "LG 트윈스", "KT 위즈", "두산 베어스",
                                         "SSG 랜더스", "NC 다이노스", "키움 히어로즈", "한화 이글스", "롯데 자이언츠"])
    }
    users.append(user)
    
users_df = pd.DataFrame(users)
print(users_df.head())


   user_id  age  gender team_preference
0        1   60    Male         기아 타이거즈
1        2   46    Male         SSG 랜더스
2        3   49  Female          두산 베어스
3        4   60    Male          LG 트윈스
4        5   20    Male          한화 이글스


In [7]:
# 상품 데이터 생성
items = []
categories = ["유니폼", "사인볼", "모자", "키링", "응원도구", "기타"]

for i in range(1, 51):  # 50개의 상품
    category = random.choice(categories)
    
    # 카테고리별 가격 범위 설정
    if category == "유니폼":
        price = random.randint(60000, 250000)  
    elif category == "사인볼":
        price = random.randint(50000, 100000)  
    elif category == "모자":
        price = random.randint(30000, 150000)  
    elif category == "키링":
        price = random.randint(10000, 30000)  
    elif category == "응원도구":
        price = random.randint(7000, 60000)  
    else:  # 기타
        price = random.randint(10000, 30000)  
    
    item = {
        "item_id": i,
        "name": f"Item {i}",
        "category": category,
        "price": price
    }
    items.append(item)

# 데이터프레임 생성
items_df = pd.DataFrame(items)
print(items_df.head())


   item_id    name category   price
0        1  Item 1      유니폼  248271
1        2  Item 2      사인볼   66544
2        3  Item 3      사인볼   97089
3        4  Item 4       모자  102465
4        5  Item 5       기타   26137


In [12]:
transactions = []

# 최근 생성된 사용자 ID (예: 90~100번 사용자)
new_user_ids = list(range(90, 101))

for i in range(1, 501):  # 500개의 거래 생성
    user_id = random.randint(1, 100)  # 사용자 ID
    item = random.choice(items)  # 상품 정보 선택 (items는 상품 데이터 리스트)
    category = item["category"]  # 상품의 카테고리
    original_price = item["price"]  # 원래 상품 가격
    
    # 카테고리별 거래 가격 설정
    if category == "유니폼":
        price = random.randint(50000, 500000) 
    elif category == "사인볼":
        price = random.randint(20000, 250000)  
    elif category == "모자":
        price = random.randint(20000, 200000)  
    elif category == "키링":
        price = random.randint(10000, 50000)  
    elif category == "응원도구":
        price = random.randint(15000, 50000)  
    else:  # 기타
        price = random.randint(10000, 300000)  
    
    # 의심 기준 초기화
    is_suspicious = 0  # 초기값: 정상 거래
    
    # 1) 가격 기준: 카테고리별 정상 가격 범위를 초과하거나 미달
    if (category == "유니폼" and (price < 100000 or price > 450000)) or \
       (category == "사인볼" and (price < 30000 or price > 200000)) or \
       (category == "모자" and (price < 25000 or price > 180000)) or \
       (category == "키링" and (price < 15000 or price > 40000)):
        is_suspicious = 1

    # 2) 비정상적인 시간대 거래
    transaction_time = faker.date_time_this_year()
    if transaction_time.hour in [3, 4]:  # 새벽 3~4시 거래
        is_suspicious = 1

    # 3) 동일 상품 반복 거래 의심
    if random.random() < 0.1:  # 10% 확률로 동일 상품 반복 거래
        is_suspicious = 1

    # 4) 비정상적인 할인/할증 비율
    if price < original_price * 0.5 or price > original_price * 2:  # 50% 이하 또는 2배 이상
        is_suspicious = 1

    # 5) 새로운 사용자
    if user_id in new_user_ids:
        is_suspicious = 1

    # 거래 생성
    transaction = {
        "transaction_id": i,
        "user_id": user_id,
        "item_id": item["item_id"],
        "category": category,
        "price": price,
        "original_price": original_price,
        "transaction_time": transaction_time,
        "is_suspicious": is_suspicious  # 사기 의심 여부
    }
    transactions.append(transaction)

# 데이터프레임 생성
transactions_df = pd.DataFrame(transactions)
print(transactions_df.head())


   transaction_id  user_id  item_id category   price  original_price  \
0               1       21        5       기타  286681           26137   
1               2       82       30      사인볼   53470           94117   
2               3       91       44       키링   25397           19619   
3               4       44       10       기타   56852           14213   
4               5       60       12       모자   42711          132935   

     transaction_time  is_suspicious  
0 2025-01-14 15:15:16              1  
1 2025-01-07 11:39:59              0  
2 2025-01-06 13:17:41              1  
3 2025-01-09 07:29:45              1  
4 2025-01-06 06:19:38              1  
